#BAD RECORD HANDLING

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/data_corrupt/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/data_corrupt/employees.csv', name='employees.csv', size=234, modificationTime=1723494503000)]

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
employees_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("salary",IntegerType(),True),
])

## Permissive Mode

In [0]:
df_data_corrupt_permissive = spark.read.format("csv").option("mode","PERMISSIVE").option("header","true").schema(employees_schema).load("dbfs:/FileStore/tables/data_corrupt/employees.csv")

df_data_corrupt_permissive.show()

+---+-------+----+------+
| id|   name| age|salary|
+---+-------+----+------+
|  1|  Alice|  29| 50000|
|  2|    Bob|null| 55000|
|  3|Charlie|  22|  null|
|  4|  David|  35|  null|
|  5|    Eve|  26|  null|
|  6|  Frank|null| 48000|
|  7|  Grace|  29| 47000|
|  8|      8|  24| 60000|
|  9|   Igor|null|   444|
| 10|   John|  28|  null|
+---+-------+----+------+



##DropMalformated Mode

In [0]:
df_data_corrupt_dropmalformed = spark.read.format("csv").option("mode","DROPMALFORMED").option("header","true").schema(employees_schema).load("dbfs:/FileStore/tables/data_corrupt/employees.csv")

df_data_corrupt_dropmalformed.show()

+---+-----+---+------+
| id| name|age|salary|
+---+-----+---+------+
|  1|Alice| 29| 50000|
|  7|Grace| 29| 47000|
|  8|    8| 24| 60000|
+---+-----+---+------+



##FailFast Mode

In [0]:
df_data_corrupt_failfast = spark.read.format("csv").option("mode","FAILFAST").option("header","true").schema(employees_schema).load("dbfs:/FileStore/tables/data_corrupt/employees.csv")

df_data_corrupt_failfast.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2192276313143937>:3
      1 df_data_corrupt_failfast = spark.read.format("csv").option("mode","FAILFAST").option("header","true").schema(employees_schema).load("dbfs:/FileStore/tables/data_corrupt/employees.csv")
----> 3 df_data_corrupt_failfast.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916         mes